# LangChain Async Example

This notebook demonstrates how to use the streaming capability of langchain and monitor the results using trulens.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
import asyncio

from langchain import LLMChain
from langchain import PromptTemplate
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models.openai import ChatOpenAI

from trulens_eval import Tru, Feedback, feedback
from trulens_eval.keys import check_keys
import trulens_eval.utils.python  # makes sure asyncio gets instrumented

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

In [ ]:
# Set up a language match feedback function.
tru = Tru()
hugs = feedback.Huggingface()
f_lang_match = Feedback(hugs.language_match).on_input_output()

# Set up an async callback.
callback = AsyncIteratorCallbackHandler()

# Setup a simple question/answer chain with streaming ChatOpenAI.
prompt = PromptTemplate.from_template("Honestly answer this question: {question}.")
llm = ChatOpenAI(
    temperature=0.0,
    streaming=True, # important
    callbacks=[callback]
)
chain = LLMChain(llm=llm, prompt=prompt)
tc = tru.Chain(chain, feedbacks=[f_lang_match])

message = "What is 1+2? Explain your answer."

# Create a task with the call to the chain, but don't wait for it yet.
f_res_record = asyncio.create_task(
    tc.acall_with_record(
        inputs=dict(question=message),
    )
)

# Instead wait for the callback's async generator, getting us each token as it comes in.
async for token in callback.aiter():
    print(token)

# By now the acall_with_record results should be ready.
res, record = await f_res_record

res

In [ ]:
# Interactive version of the above.

from ipywidgets import widgets, interact
from IPython.display import clear_output, display
import asyncio

out = widgets.HTML(value="")
display(out)

async def process_question(message):
    if message == "":
        return

    out.value = message + "<br/><br/>"

    # Set up an async callback.
    callback = AsyncIteratorCallbackHandler()

    # Setup a simple question/answer chain with streaming ChatOpenAI.
    prompt = PromptTemplate.from_template("Honestly answer this question: {question}.")
    llm = ChatOpenAI(
        temperature=0.0,
        streaming=True, # important
        callbacks=[callback]
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tc = tru.Chain(chain, feedbacks=[f_lang_match])

    f_res_record = asyncio.create_task(
        tc.acall_with_record(
            inputs=dict(question=message),
        )
    )
    
    # Instead wait for the callback's async generator, getting us each token as it comes in.
    async for token in callback.aiter():
        out.value += token

    # By now the acall_with_record results should be ready.
    res, record = await f_res_record

    print(res)

@interact(message=widgets.Text(continuous_update=False))
def ask_question(message: str):
    asyncio.ensure_future(process_question(message))